In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras import Model, Sequential, layers
from tensorflow.keras.layers import (LSTM, Concatenate, Dense, Input, Maximum,
                                     Multiply, Reshape, Subtract,
                                     TimeDistributed)
from tensorflow.keras.utils import plot_model

# https://www.tensorflow.org/guide/keras/train_and_evaluate
# https://www.tensorflow.org/tutorials/structured_data/feature_columns

In [ ]:
memory_size = 20  # 20, 40, 60
time_scale = 3600

In [ ]:
inputLayer = Input(shape=(4, 1), name='input')
cpu = Dense(time_scale, name='CPU')(inputLayer)
cpu_lstm = LSTM(memory_size)(cpu)
memory = Dense(time_scale, name='Memory')(inputLayer)
memory_lstm = LSTM(memory_size)(memory)

mul1 = Multiply()([cpu_lstm, memory_lstm])

allocated0 = Dense(time_scale, name='Allocated')(mul1)
allocated = Reshape((time_scale, 1), input_shape=(time_scale,))(allocated0)
allocated_lstm = LSTM(memory_size)(allocated)
used0 = Dense(time_scale, name='Used')(mul1)
used = Reshape((time_scale, 1), input_shape=(time_scale,))(used0)
used_lstm = LSTM(memory_size)(used)

mul2 = Multiply()([allocated_lstm, used_lstm])

dense1 = Dense(time_scale, name='Dense1')(mul2)
dense2 = Dense(time_scale, name='Dense2')(mul2)

model = Model(inputs=[inputLayer], outputs=[dense1, dense2])

In [ ]:
X_df = pd.read_csv('./datasets/standardized_feature_df.csv', index_col=0)
X_df

In [ ]:
y_df = pd.read_csv('./datasets/normalized_label_df.csv', index_col=0)
y_df

In [ ]:
from pandas import DataFrame


def df_to_dataset(feature_df: DataFrame, label_df: DataFrame, shuffle: bool = True, batch_size: int = 32):
    # feature_tensor = tf.convert_to_tensor(feature_df)
    # label_tensor = tf.convert_to_tensor(label_df)
    # feature_df = feature_df.copy()
    # label_df = label_df.copy()
    # dataset = tf.data.Dataset.from_tensor_slices((feature_tensor, label_tensor))
    
    dataset = tf.data.Dataset.from_tensor_slices((dict(feature_df), dict(label_df)))
    
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(dataset))
    dataset = dataset.batch(batch_size)
    return dataset
    

In [ ]:
train_dataset = df_to_dataset(X_df, y_df)
train_dataset

In [ ]:
for feature_batch, label_batch in train_dataset.take(1):
    print(f'Every feature: {feature_batch.keys()}')
    print(f'Every label: {label_batch.keys()}')
    
feature_batch, label_batch in train_dataset.take(1)

In [ ]:
X_tensor = tf.convert_to_tensor(X_df)
y_tensor = tf.convert_to_tensor(y_df)

In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.MAE)
model.fit(X_tensor, y_tensor, epochs=3, batch_size=32)

### Tensorflow Error Output

![tf-error](./figures/tensorflow_error_output.png)